In [ ]:
!pip install boilerpy3

In [ ]:
import json
import re
import requests
from boilerpy3 import extractors

In [ ]:
#open the file

f = open("topics.json", encoding='utf8')
topics = json.load(f)


In [ ]:
total_ergebnisse = []

web_ergebnisse = []

target_uri_ergebnisse = []


In [ ]:
#preprocess the queries

for i in range(len(topics)):
    topics[i]['title'] = topics[i]['title'].replace(' ', ' AND ')
    topics[i]['title'] = re.sub('[?:,]', '', topics[i]['title'])

topics

In [ ]:
attr = {"apikey": "7dd15626-53aa-46c6-bd34-b2feaa2d9d81",
        "query": "",
        "index": "cw12",
        "size": 50
        }

In [ ]:
#send queries to ChatNoir and collect the number of results and some information (score,uuid,trec_id,......)

for q in topics:
    attr["query"] = q['title']
    url = "https://www.chatnoir.eu/api/v1/_search?"
    r = requests.get(url, attr)
    
    total_ergebnisse.append(r.json()['meta']['total_results'])
    
    for i in range(len(r.json()['results'])):
        
        score = r.json()['results'][i]['score']
        uuid = r.json()['results'][i]['uuid']
        index = r.json()['results'][i]['index']
        trec_id = r.json()['results'][i]['trec_id']
        target_hostname = r.json()['results'][i]['target_hostname']
        page_rank = r.json()['results'][i]['page_rank']
        spam_rank = r.json()['results'][i]['spam_rank']
        web_ergebnisse.append((score,uuid,index, trec_id,target_hostname,page_rank,spam_rank))
        
        target_uri = r.json()['results'][i]['target_uri']
        target_uri_ergebnisse.append(target_uri)
    

In [ ]:
total_ergebnisse


In [ ]:
web_ergebnisse

In [ ]:
target_uri_ergebnisse

In [ ]:
#extract the contents of the first 50 web page for each query using Boilerpipe
#save the results in demofile2.txt

extractor = extractors.ArticleExtractor()

f = open("demofile2.txt", "w", encoding='utf8')
for i in target_uri_ergebnisse:
    try:
        resp = requests.get(i)
        content = extractor.get_content(resp.text)
        f.write(content + "\n"+ "\n"+ "\n"+ "\n"+ "\n"+ "\n" )
    except requests.exceptions.RequestException as e:
        pass
    except AttributeError:
        pass
    except IndexError:
        pass

f.close()